# Module 1: E-commerce Data Exploration

**Objective**: Explore and understand the raw e-commerce dataset to inform cleaning strategies.

**Dataset**: Online Retail dataset with transaction records including:
- InvoiceNo: Transaction identifier
- StockCode: Product identifier
- Description: Product description
- Quantity: Number of items purchased
- UnitPrice: Price per item
- CustomerID: Customer identifier
- Country: Customer location

**Expected Outcome**: Identify data quality issues and design appropriate cleaning strategy.

## 1. Initial Data Loading

Load the raw dataset and examine its structure.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load with correct encoding
df = pd.read_csv("data/raw/dataset.csv", encoding="ISO-8859-1")

print(f"Dataset shape: {df.shape}")
print(f"Total rows: {len(df):,}")
df.head()

## 2. Data Quality Assessment

Check for missing values, data types, and potential issues.

In [ ]:
# Basic info
print("\nDataset Info:")
df.info()

# Missing values
print("\nMissing Values:")
missing = df.isna().sum()
print(missing[missing > 0])

# Missing percentages
print("\nMissing Percentages:")
missing_pct = (df.isna().sum() / len(df) * 100).round(2)
print(missing_pct[missing_pct > 0])

### Key Observations:
- **CustomerID**: High percentage of missing values (~25%)
- **Description**: Some missing product descriptions
- These missing values will need to be handled during cleaning

**Decision**: Will remove rows with missing critical fields (Description, StockCode) but keep CustomerID optional.

## 3. Data Corruption Analysis

Check for special characters and corrupted data in text fields.

In [ ]:
# Sample StockCode values
print("Sample StockCode values (including potential corruption):")
print(df['StockCode'].value_counts().head(20))

# Check for special characters
special_chars = df['StockCode'].astype(str).str.contains(r'[^A-Za-z0-9]', regex=True)
print(f"\nStockCodes with special characters: {special_chars.sum():,}")

# Sample corrupted values
print("\nExamples of corrupted StockCodes:")
print(df[special_chars]['StockCode'].head(10))

### Key Findings:
- StockCode contains special characters that need cleaning
- Some codes have symbols like Ã, ö, ^ that indicate encoding issues

**Solution**: Apply regex cleaning to remove all non-alphanumeric characters

## 4. Test Cleaning Functions

Test regex patterns for data cleaning before implementing in production.

In [ ]:
import re

def clean_stockcode(x):
    """Remove non-alphanumeric characters from StockCode"""
    if pd.isna(x):
        return x
    return re.sub(r'[^A-Za-z0-9]', '', str(x))

# Test on sample
test_codes = df['StockCode'].head(100)
cleaned_codes = test_codes.apply(clean_stockcode)

print("Before and After Cleaning:")
comparison = pd.DataFrame({
    'Original': test_codes,
    'Cleaned': cleaned_codes
})
print(comparison[comparison['Original'] != comparison['Cleaned']].head(10))

## 5. Numeric Field Analysis

Examine Quantity and UnitPrice for invalid values.

In [ ]:
# Quantity analysis
print("Quantity Statistics:")
print(df['Quantity'].describe())

# Check for negative quantities
negative_qty = df['Quantity'] < 0
print(f"\nNegative quantities: {negative_qty.sum():,} ({negative_qty.sum()/len(df)*100:.2f}%)")

# Price analysis
print("\nUnitPrice Statistics:")
print(df['UnitPrice'].describe())

# Check for zero/negative prices
invalid_price = df['UnitPrice'] <= 0
print(f"\nInvalid prices (≤0): {invalid_price.sum():,} ({invalid_price.sum()/len(df)*100:.2f}%)")

### Key Insights:
- Negative quantities likely represent returns/cancellations
- Zero prices are data quality issues

**Decision**: Filter to keep only Quantity > 0 and UnitPrice > 0 for valid transactions

## 6. Unique Products Analysis

Understand the distribution of unique products for vector database.

In [ ]:
# Count unique products
unique_products = df['StockCode'].nunique()
print(f"Unique StockCodes: {unique_products:,}")

# Check descriptions per StockCode
desc_per_code = df.groupby('StockCode')['Description'].nunique()
print(f"\nStockCodes with multiple descriptions: {(desc_per_code > 1).sum():,}")

# Top products by transaction count
print("\nTop 10 Products by Transaction Count:")
top_products = df['StockCode'].value_counts().head(10)
print(top_products)

## 7. Summary and Next Steps

### Data Quality Issues Identified:
1. Special characters in StockCode (encoding corruption)
2. Missing values in CustomerID (~25%) and Description
3. Negative quantities (returns)
4. Zero/negative prices
5. Some StockCodes have multiple descriptions

### Cleaning Strategy:
1. ✅ Remove special characters from all text fields
2. ✅ Filter Quantity > 0 and UnitPrice > 0
3. ✅ Remove rows with empty critical fields
4. ✅ Group by StockCode and take first description
5. ✅ Remove duplicates

### Expected Outcome:
- Reduction from 541,910 to ~400,000 rows (26% removal)
- Clean dataset ready for embedding generation
- ~3,600 unique products for vector database

**Next**: Implement cleaning in `data_cleaner.py` service